In [1]:
# Trích xuất dữ liệu từ Amazon
# import các file cần thiết
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


driver = webdriver.Chrome(executable_path=r"C:\Users\maico\Documents\Python Scripts\chromedriver\chromedriver.exe")


# Viết chương trình lấy url bằng item và số trang 
def get_url(search_terms,page):
    template1 = "https://www.amazon.com/s?k=" 
    template2 = "&rh=n%3A1292115011"
    template3 = "&ref=nb_sb_noss"
    search_terms = search_terms.replace(" ","+")
    return template1 + search_terms + template2 + "&page=" + str(page) + template3
def extract_record(item):
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com"+atag.get("href")
    
    # price
    try: 
        price_parent=item.find("span","a-price")
        price = price_parent.find("span","a-offscreen").text
    except AttributeError:
        return 
    
    # rank and rating 
    try:
        rating = item.i.text
        review_count = item.find("span",{"class":"a-size-base"}).text
    except AttributeError:
        rating = ""
        review_count = ""
    
    result = (description, price, rating, review_count, url)
    return result
def final(search_item):
    records = []
    for i in range(1,17):
        url = get_url(search_item,i)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        results = soup.find_all("div",{"data-component-type":"s-search-result"})
        for item in results:
            t = extract_record(item)
            records.append(t)
    return records
final_result = final("ultrawide monitor")
final_result[0]

        
    
    

('AOC CQ34G2 Super Curved Frameless Gaming Monitor, UltraWide FHD 2560x1080, 1500R VA Panel, 1ms MPRT, 75Hz, FreeSync, Height Adjustable, 3-Yr Zero Dead Pixels Black/Silver',
 '$299.40',
 '4.7 out of 5 stars',
 '8,256',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_computers_sr_pg1_1?ie=UTF8&adId=A0017644VP076XASNW48&url=%2FAOC-CQ34G2-Frameless-UltraWide-Adjustable%2Fdp%2FB08632C4MS%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1620226041%26s%3Dpc%26sr%3D1-1-spons%26psc%3D1&qualifier=1620226041&id=7472582927744874&widgetName=sp_atf')

In [2]:
len(final_result)

391

In [4]:
# Xuất kết quả ra file
import pandas as pd 
df = pd.DataFrame(final_result,columns = ["description", "price", "rating", "review_count", "url"])
df.to_csv("amazon.csv",index=False)